# Formating Genre 

In [2]:
import pickle 
import pandas as pd

In [1]:
# scratch_path = 'home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/'
scratch_path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Filtered/'

In [3]:
# train_p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/train_val4.pkl'
#scratch_path + 'train_val_data2.pkl'

train_p = scratch_path + 'train_val_with_appear.pkl'

In [4]:
data = pickle.load(open(train_p, "rb"))

In [5]:
tracks = data['df_track']
test_set = data['test_set']

In [10]:
# tracks = tracks[(tracks.tid.isin(interact.tid.unique())) | (tracks.tid.isin(test_set.tid.unique()))]

In [6]:
tracks.columns

Index(['old_tid', 'track_uri', 'track_name', 'album_uri', 'album_name',
       'primary_artist', 'artist_names', 'artist_uris', 'track_name_emb',
       'img_emb', 'genres', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'og_artist_name', 'og_track_name',
       'album_popularity', 'popularity', 'tid', 'genres_vec', 'appear_raw',
       'appear_cat2', 'appear_pop2', 'appear_norm', 'danceability_10cat',
       'energy_10cat', 'loudness_10cat', 'speechiness_10cat',
       'acousticness_10cat', 'instrumentalness_10cat', 'liveness_10cat',
       'valence_10cat', 'tempo_10cat', 'primary_artist_uri',
       'primary_artist_name', 'arid'],
      dtype='object')

In [7]:
len(tracks)

337002

In [8]:
# tracks

In [9]:
tracks['genres']

0         [australian pop]
1         [australian pop]
2         [australian pop]
3         [australian pop]
4         [australian pop]
                ...       
336997       [synthesizer]
336998         [spacewave]
336999                  []
337000                  []
337001                  []
Name: genres, Length: 337002, dtype: object

In [10]:
tracks['genres'].iloc[0]

['australian pop']

In [11]:
all_genre_vectors = tracks['genres'].tolist()
all_genres = set([g for vec in all_genre_vectors for g in vec]) 

In [12]:
genre_mapping = dict(zip(all_genres, list(range(1, len(all_genres)+1))))

In [13]:
tracks['genres_vec'] = tracks['genres'].apply(lambda x: [genre_mapping[g] for g in x])

In [14]:
tracks[['genres', 'genres_vec']]

,genres,genres_vec
0,[australian pop],[747]
1,[australian pop],[747]
2,[australian pop],[747]
3,[australian pop],[747]
4,[australian pop],[747]
...,...,...
336997,[synthesizer],[1097]
336998,[spacewave],[1714]
336999,[],[]
337000,[],[]


In [15]:
def fill(x): 
    if len(x) == 0: 
        return [0]
    else: return x 
    
tracks['genres_vec'] = tracks['genres_vec'].apply(lambda x: fill(x))

In [16]:
tracks[['tid',  'genres', 'genres_vec']] #'old_tid',

,tid,genres,genres_vec
0,0,[australian pop],[747]
1,1,[australian pop],[747]
2,2,[australian pop],[747]
3,3,[australian pop],[747]
4,4,[australian pop],[747]
...,...,...,...
336997,336997,[synthesizer],[1097]
336998,336998,[spacewave],[1714]
336999,336999,[],[0]
337000,337000,[],[0]


# Appearance

In [17]:
import pickle 
import pandas as pd

In [12]:
# train_p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/train_val4.pkl'
# scratch_path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/'
# train_p = scratch_path + 'train_val_data6.pkl'


In [13]:
data = pickle.load(open(train_p, "rb"))

In [14]:
tracks = data['df_track']
test_set = data['test_set']

In [12]:
interact = data['df_playlist']

In [19]:
tid_vis = interact.groupby('tid')['pid'].apply(len).reset_index(name='appear_raw').sort_values('tid') #'old_tid' for subset
tid_vis

,tid,appear_raw
0,0,778
1,1,58
2,2,4
3,3,28
4,4,9
...,...,...
336997,336997,9
336998,336998,12
336999,336999,10
337000,337000,29


In [20]:
tracks_with_appear = pd.merge(tracks, tid_vis, how='left')

In [21]:
bins = 2
tracks_with_appear['appear_cat{}'.format(bins)] = pd.cut(tracks_with_appear['appear_raw'], bins, labels=list(range(bins)), retbins = False, precision=3, ordered=True)

In [22]:
tracks_with_appear['appear_pop{}'.format(bins)] = tracks_with_appear['appear_cat{}'.format(bins)]

In [23]:
tracks_with_appear['appear_norm'] = tracks_with_appear['appear_raw']/len(interact)

In [29]:
tracks_with_appear = tracks_with_appear.fillna(0)

In [24]:
tracks_with_appear

,old_tid,track_uri,track_name,album_uri,album_name,primary_artist,artist_names,artist_uris,track_name_emb,img_emb,...,og_artist_name,og_track_name,album_popularity,popularity,tid,genres_vec,appear_raw,appear_cat2,appear_pop2,appear_norm
0,0,4wCmqSrbyCgxEXROQE6vtV,Somebody That I Used To Know,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,"[Gotye, Kimbra]","[spotify:artist:2AsusXITU8P25dlRNhcAbG, spotif...","[0.021594396, -0.006500091, 0.0013173159, 0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,Gotye,Somebody That I Used to Know,67,2339051,0,[747],778,0,0,6.230421e-05
1,1,6h2q5GhVc9itEB6NqOEMd4,Eyes Wide Open,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0037547485, 0.0038352187, -0.07585667, 0.0...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,Gotye,Eyes Wide Open,67,2339051,1,[747],58,0,0,4.644787e-06
2,2,2WVvmWnQeEkLAtWHkJm4j7,Making Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0449905, 0.00800193, -0.08014139, 0.038365...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,Gotye,Making Mirrors,67,2339051,2,[747],4,0,0,3.203301e-07
3,3,0wm2hhtpRDtidodGobkttG,Smoke And Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.046797097, 0.01333951, -0.04641572, -0.000...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,Gotye,Smoke and Mirrors,67,2339051,3,[747],28,0,0,2.242311e-06
4,4,6l8AiDMKX10Nu2IZJcPb4i,In Your Light,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.030510716, -0.08082959, -0.04938146, -0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,Gotye,In Your Light,67,2339051,4,[747],9,0,0,7.207428e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336997,1269703,5lsR1F6NGUYzuIbMxF4lY2,Forbidden Colours,spotify:album:3wd4ials0jGsnSBoba3keA,Sound Of The Movies,Star Inc.,[Star Inc.],[spotify:artist:7uxS87ca4fSZmbqAfAZHgQ],"[0.055395022, 0.009642398, -0.039647937, 0.051...","[0.43341178, 2.094814, 0.13000806, 0.008512065...",...,Star Inc.,Forbidden Colours,21,1331,336997,[1097],9,0,0,7.207428e-07
336998,1269705,7kCPTBNgbcgbqiwzX12FGy,Forever Linda Lee,spotify:album:7uCJGpWTVHQGOZqWhUI1jW,Forever Linda Lee,PatternShift,[PatternShift],[spotify:artist:6MNFOvQOGj0BYmHJa6tVIV],"[0.056932643, 0.050643943, -0.009315571, 0.017...","[0.014573085, 1.433477, 0.12552735, 0.4734528,...",...,PatternShift,Forever Linda Lee,6,4049,336998,[1714],12,0,0,9.609904e-07
336999,1269707,3hNvKL0Xvw9ocyZnOcKgiR,Free Bass - Joey Negro Funk Equation Mix,spotify:album:7Exa2bP9PDIK95RIs1U49A,Space Ibiza: 1989 - 2016,Joey Negro,[Joey Negro],[spotify:artist:6JhuegEmpyKFXAH3cgUv52],"[0.020323858, 0.0072746, -0.0089004, -0.010807...","[0.38741714, 0.74071234, 0.6717084, 0.0, 0.803...",...,Joey Negro,Free Bass - Joey Negro Funk Equation Mix,0,94,336999,[0],10,0,0,8.008254e-07
337000,1269719,2R92Lq8g6bgcBAd8VFjQoF,Giants,spotify:album:5gqtkubVUxFQjzVBJRG6Ru,Giants,Jackson Guthy,[Jackson Guthy],[spotify:artist:5n4GpmlM9nTuOiMWpQhHKo],"[-0.039403204, 0.0071366094, -0.059462447, -0....","[0.41514772, 0.98289704, 0.43613452, 0.0291842...",...,Jackson Guthy,Giants,0,12311,337000,[0],29,0,0,2.322394e-06


In [25]:
data.keys()

dict_keys(['df_track', 'df_playlist_info', 'df_playlist', 'train_indices', 'val_indices', 'test_set'])

# 80/20 LT split

In [1]:
import pickle
import pandas as pd 

In [2]:
scratch_path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/'
train_path=scratch_path + 'LFM_Subset/Smaller_Size/train_val_data8.pkl'

data = pickle.load(open(train_path, "rb"))
tracks = data['df_track']

In [3]:
tracks_sort_by_appear = tracks.sort_values('appear_raw', ascending=False).reset_index(drop=True)

In [4]:
LT_cutoff = int(float(len(tracks_sort_by_appear))*0.2)
print(LT_cutoff, len(tracks_sort_by_appear)) 
tracks_sort_by_appear.iloc[:LT_cutoff, :][['tid', 'appear_raw']]

178113 890568


,tid,appear_raw
0,7495,25885
1,31133,13596
2,223089,9608
3,137432,7378
4,92291,6050
...,...,...
178108,573397,8
178109,599117,8
178110,426537,8
178111,627664,8


In [5]:
cutoff_val = tracks_sort_by_appear.iloc[LT_cutoff, :].appear_raw

tracks['80_20_LT'] = tracks['appear_raw'].apply(lambda x: 0 if x < cutoff_val else 1 )


In [6]:
tracks

,old_tid,track_uri,track_name,album_uri,album_name,primary_artist,artist_names,artist_uris,track_name_emb,img_emb,...,#pids_norm,pop_by_pid,appear_cat10,appear_pop10,appear_cat5,appear_pop5,appear_cat2,appear_pop2,log10_popcat,80_20_LT
0,0,4wCmqSrbyCgxEXROQE6vtV,Somebody That I Used To Know,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,"[Gotye, Kimbra]","[spotify:artist:2AsusXITU8P25dlRNhcAbG, spotif...","[0.021594396, -0.006500091, 0.0013173159, 0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,0.021934,0.978066,0,0,0,0,0,0,5,1
1,1,6h2q5GhVc9itEB6NqOEMd4,Eyes Wide Open,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0037547485, 0.0038352187, -0.07585667, 0.0...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,0.002066,0.997934,0,0,0,0,0,0,3,1
2,2,2WVvmWnQeEkLAtWHkJm4j7,Making Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0449905, 0.00800193, -0.08014139, 0.038365...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,0.000393,0.999607,0,0,0,0,0,0,1,0
3,3,0wm2hhtpRDtidodGobkttG,Smoke And Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.046797097, 0.01333951, -0.04641572, -0.000...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,0.001082,0.998918,0,0,0,0,0,0,2,1
4,4,6l8AiDMKX10Nu2IZJcPb4i,In Your Light,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.030510716, -0.08082959, -0.04938146, -0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,0.000590,0.999410,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890563,1269720,7rWR2e3fYmAolBRW98SCLF,Give Into,spotify:album:4WW3qyZJLHZwQ31IZrlEYi,Give Into,Nutronic,[NUTRONIC],[spotify:artist:3VnPsIJuT0zVGQPJk2MaRr],"[-0.0011738086, -0.076979466, -0.07855234, 0.0...","[0.071334325, 0.22042117, 0.13449018, 0.638719...",...,0.000197,0.999803,0,0,0,0,0,0,1,0
890564,1269721,4F1NlboTmnSzkOLv1OEYwl,Give You More,spotify:album:4lhkzxbknT9ETM2p7yi39x,Give You More EP,HIGHSOCIETY,[HIGHSOCIETY],[spotify:artist:32MQN2sITkJGNQN40QLPEa],"[0.05221358, -0.038427457, -0.008215231, 0.004...","[0.13059509, 1.2819605, 0.34352416, 0.4384208,...",...,0.000295,0.999705,0,0,0,0,0,0,1,0
890565,1269722,5Wakl6FV8JtagD7GZqWuci,Golden Hour,spotify:album:3WDoeMOzd5rEogwr1J7P5E,Give Me Some Time,Matthew Pinder,[Matthew Pinder],[spotify:artist:7vdNkYg52hwaThO9MOX5vv],"[-0.0211157, 0.030536134, -0.013183946, 0.0160...","[0.02851578, 0.72624457, 0.8418279, 0.39861402...",...,0.000197,0.999803,0,0,0,0,0,0,1,0
890566,1269723,096qaB1U1wjnVYP9e0Ql3p,God Of Mercy And Compassion,spotify:album:0hoXR1vY2m8RkySQTMvAni,Lent At Ephesus,"Benedictines Of Mary, Queen Of Apostles","[Benedictines of Mary, Queen of Apostles]",[spotify:artist:4o79bJJ2tBszgEDEvLhB2L],"[0.0057596755, -0.019511785, 0.014406702, 0.00...","[0.38570824, 2.065993, 1.3967322, 0.30380917, ...",...,0.000098,0.999902,0,0,0,0,0,0,0,0


In [7]:
save = {
    'df_track': tracks, 
    'df_playlist': data['df_playlist'], 
    'df_playlist_info': data['df_playlist_info'], 
    'test_set': data['test_set'], 
    'train_indices': data['train_indices'], 
    'val_indices': data['val_indices']
}

In [9]:
output_path = scratch_path + 'LFM_Subset/Smaller_Size/train_val_data9.pkl'
output_path

'/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data9.pkl'

In [10]:
pickle.dump(save, open(output_path, "wb"))


# Make music values into 10 bins 


In [28]:
mus_feat = ['danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']

In [29]:
for m in mus_feat: 
    tracks_with_appear[m+'_10cat'] = pd.cut(tracks_with_appear[m], 10, 
       labels=list(range(10)), retbins = False, precision=3, ordered=True)
    
tracks_with_appear

,old_tid,track_uri,track_name,album_uri,album_name,primary_artist,artist_names,artist_uris,track_name_emb,img_emb,...,appear_norm,danceability_10cat,energy_10cat,loudness_10cat,speechiness_10cat,acousticness_10cat,instrumentalness_10cat,liveness_10cat,valence_10cat,tempo_10cat
0,0,4wCmqSrbyCgxEXROQE6vtV,Somebody That I Used To Know,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,"[Gotye, Kimbra]","[spotify:artist:2AsusXITU8P25dlRNhcAbG, spotif...","[0.021594396, -0.006500091, 0.0013173159, 0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,6.230421e-05,8,4,8,0,5,0,0,7,5
1,1,6h2q5GhVc9itEB6NqOEMd4,Eyes Wide Open,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0037547485, 0.0038352187, -0.07585667, 0.0...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,4.644787e-06,6,8,8,0,1,0,1,6,5
2,2,2WVvmWnQeEkLAtWHkJm4j7,Making Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.0449905, 0.00800193, -0.08014139, 0.038365...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,3.203301e-07,2,0,6,0,4,8,1,0,2
3,3,0wm2hhtpRDtidodGobkttG,Smoke And Mirrors,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.046797097, 0.01333951, -0.04641572, -0.000...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,2.242311e-06,3,8,8,1,1,1,2,6,7
4,4,6l8AiDMKX10Nu2IZJcPb4i,In Your Light,spotify:album:1HjSyGjmLNjRAKgT9t1cna,Making Mirrors,Gotye,[Gotye],[spotify:artist:2AsusXITU8P25dlRNhcAbG],"[-0.030510716, -0.08082959, -0.04938146, -0.00...","[0.002694027, 0.8552622, 0.6958655, 0.01704761...",...,7.207428e-07,6,9,8,0,0,0,1,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336997,1269703,5lsR1F6NGUYzuIbMxF4lY2,Forbidden Colours,spotify:album:3wd4ials0jGsnSBoba3keA,Sound Of The Movies,Star Inc.,[Star Inc.],[spotify:artist:7uxS87ca4fSZmbqAfAZHgQ],"[0.055395022, 0.009642398, -0.039647937, 0.051...","[0.43341178, 2.094814, 0.13000806, 0.008512065...",...,7.207428e-07,6,5,7,0,4,6,0,4,4
336998,1269705,7kCPTBNgbcgbqiwzX12FGy,Forever Linda Lee,spotify:album:7uCJGpWTVHQGOZqWhUI1jW,Forever Linda Lee,PatternShift,[PatternShift],[spotify:artist:6MNFOvQOGj0BYmHJa6tVIV],"[0.056932643, 0.050643943, -0.009315571, 0.017...","[0.014573085, 1.433477, 0.12552735, 0.4734528,...",...,9.609904e-07,5,6,7,0,0,9,1,0,5
336999,1269707,3hNvKL0Xvw9ocyZnOcKgiR,Free Bass - Joey Negro Funk Equation Mix,spotify:album:7Exa2bP9PDIK95RIs1U49A,Space Ibiza: 1989 - 2016,Joey Negro,[Joey Negro],[spotify:artist:6JhuegEmpyKFXAH3cgUv52],"[0.020323858, 0.0072746, -0.0089004, -0.010807...","[0.38741714, 0.74071234, 0.6717084, 0.0, 0.803...",...,8.008254e-07,8,8,8,0,0,9,0,6,4
337000,1269719,2R92Lq8g6bgcBAd8VFjQoF,Giants,spotify:album:5gqtkubVUxFQjzVBJRG6Ru,Giants,Jackson Guthy,[Jackson Guthy],[spotify:artist:5n4GpmlM9nTuOiMWpQhHKo],"[-0.039403204, 0.0071366094, -0.059462447, -0....","[0.41514772, 0.98289704, 0.43613452, 0.0291842...",...,2.322394e-06,4,6,8,0,0,0,1,1,6


### Load ARIDS

In [30]:
tracks_with_appear['primary_artist_uri'] = tracks['artist_uris'].apply(lambda x: x[0])
tracks_with_appear['primary_artist_name'] = tracks['artist_names'].apply(lambda x: x[0])

In [31]:
all_artist_vectors5 = tracks_with_appear['primary_artist_uri'].tolist()
all_artists5 = set(all_artist_vectors5)
arid_mapping = dict(zip(all_artists5, list(range(len(all_artists5)))))

In [32]:
tracks_with_appear['arid'] = tracks_with_appear['primary_artist_uri'].apply(lambda x: arid_mapping[x])

In [8]:
tracks['80_20_LT'] = tracks['appear_raw'].apply(lambda x: 0 if x < 42 else 1)

In [10]:
mod_data = {
#     'df_track': tracks_with_appear,
    'df_track': tracks, 
    'df_playlist': data['df_playlist'], # interact, 
    'df_playlist_info': data['df_playlist_info'], 
    'train_indices': data['train_indices'], 
    'val_indices': data['val_indices'], 
    'test_set': data['test_set']
}

In [11]:
# p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/train_val5.pkl'
p = scratch_path + 'train_val_with_appear2.pkl'
p

'/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Filtered/train_val_with_appear2.pkl'

In [12]:
pickle.dump(mod_data, open(p, "wb"))

In [35]:
tracks_with_appear.columns

Index(['old_tid', 'track_uri', 'track_name', 'album_uri', 'album_name',
       'primary_artist', 'artist_names', 'artist_uris', 'track_name_emb',
       'img_emb', 'genres', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'og_artist_name', 'og_track_name',
       'album_popularity', 'popularity', 'tid', 'genres_vec', 'appear_raw',
       'appear_cat', 'appear_pop', 'appear_norm', 'danceability_10cat',
       'energy_10cat', 'loudness_10cat', 'speechiness_10cat',
       'acousticness_10cat', 'instrumentalness_10cat', 'liveness_10cat',
       'valence_10cat', 'tempo_10cat', 'primary_artist_uri',
       'primary_artist_name', 'arid'],
      dtype='object')

In [36]:
tracks_with_appear['tid']

0              585
1           183293
2           183024
3            54249
4            14392
            ...   
1134697     943855
1134698     454818
1134699    1026581
1134700    1009961
1134701    1028758
Name: tid, Length: 1134702, dtype: int64

# Popularity norm 

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data5.pkl'
path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/train_val5.pkl'

In [3]:
data = pickle.load(open(path, "rb"))

In [4]:
tracks = data['df_track']
users  = data['df_playlist_info']
interact = data['df_playlist']
test_set = data['test_set']

In [5]:
len(tracks)

1134702

In [6]:
tracks[~(tracks.tid.isin(interact.tid.unique())) & (tracks.tid.isin(test_set.tid.unique())) ]

,old_tid,track_uri,track_name,album_uri,album_name,primary_artist,artist_names,artist_uris,track_name_emb,img_emb,...,loudness_10cat,speechiness_10cat,acousticness_10cat,instrumentalness_10cat,liveness_10cat,valence_10cat,tempo_10cat,primary_artist_uri,primary_artist_name,arid


In [7]:
pid_in_tid = interact.groupby(['tid'])['pid'].apply(list).reset_index(name='#pids').sort_values('#pids')
pid_in_tid['#pids_unique'] = pid_in_tid['#pids'].apply(lambda x: len(set(x)))
pid_in_tid['#pids_norm'] = pid_in_tid['#pids_unique']/len(interact.pid.unique())
pid_in_tid['pop_by_pid'] = pid_in_tid['#pids_norm'].apply(lambda x: 1 - x)

In [8]:
pid_in_tid

,tid,#pids,#pids_unique,#pids_norm,pop_by_pid
750323,750323,[0],1,0.000092,0.999908
221314,221314,[0],1,0.000092,0.999908
1068261,1068261,[0],1,0.000092,0.999908
749809,749809,[0],1,0.000092,0.999908
938448,938448,[0],1,0.000092,0.999908
...,...,...,...,...,...
1132377,1132377,[10828],1,0.000092,0.999908
1134114,1134114,[10831],1,0.000092,0.999908
1134566,1134566,[10833],1,0.000092,0.999908
1134456,1134456,[10833],1,0.000092,0.999908


In [9]:
tracks = pd.merge(tracks, pid_in_tid)

In [10]:
len(tracks), len(tracks.dropna())

(1134702, 1134702)

In [12]:
mod_data = {
    'df_track': tracks, 
    'df_playlist': interact, 
    'df_playlist_info': data['df_playlist_info'], 
    'train_indices': data['train_indices'], 
    'val_indices': data['val_indices'], 
    'test_set': data['test_set']
}

In [13]:
# p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data6.pkl'
p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Full_Size/train_val6.pkl'

In [ ]:
pickle.dump(mod_data, open(p, "wb"))

# Check 

In [10]:
import pickle 
import pandas as pd

p = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data8.pkl'

In [2]:
data1 = pickle.load(open(p, "rb"))

In [3]:
interact_full = data1['df_playlist'] 
tracks_full = data1['df_track']

In [12]:
tracks_full.columns, 'av_pop' in tracks_full.columns

(Index(['old_tid', 'track_uri', 'track_name', 'album_uri', 'album_name',
        'primary_artist', 'artist_names', 'artist_uris', 'track_name_emb',
        'img_emb', 'genres', 'danceability', 'energy', 'loudness',
        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
        'valence', 'tempo', 'og_artist_name', 'og_track_name',
        'album_popularity', 'popularity', 'tid', 'genres_vec', 'appear_raw',
        'appear_cat', 'appear_pop', 'appear_norm', 'danceability_10cat',
        'energy_10cat', 'loudness_10cat', 'speechiness_10cat',
        'acousticness_10cat', 'instrumentalness_10cat', 'liveness_10cat',
        'valence_10cat', 'tempo_10cat', 'primary_artist_uri',
        'primary_artist_name', 'arid', '#pids', '#pids_unique', '#pids_norm',
        'pop_by_pid', 'appear_cat10', 'appear_pop10', 'appear_cat5',
        'appear_pop5', 'appear_cat2', 'appear_pop2', 'log10_popcat'],
       dtype='object'),
 False)

In [48]:
import numpy as np
bins = 10
tracks_full = tracks_full[tracks_full['appear_raw']>= 1]
tracks_full['log10_popcat'] = pd.cut(np.log10(tracks_full['appear_raw']), bins, labels=list(range(bins)), retbins = False, precision=3, ordered=True)


In [9]:
bins = 10 
for i in range(bins): 
    print("Bin:{}, Track Count (by log smoothed appearance):{}, Track Count (by raw appearance):{}".format(
          i, len(tracks_full[tracks_full['log10_popcat'] == i]), len(tracks_full[tracks_full['appear_pop10'] == i])))
    

Bin:0, Track Count (by log smoothed appearance):450230, Track Count (by raw appearance):890554
Bin:1, Track Count (by log smoothed appearance):246159, Track Count (by raw appearance):9
Bin:2, Track Count (by log smoothed appearance):127044, Track Count (by raw appearance):2
Bin:3, Track Count (by log smoothed appearance):49531, Track Count (by raw appearance):1
Bin:4, Track Count (by log smoothed appearance):14110, Track Count (by raw appearance):0
Bin:5, Track Count (by log smoothed appearance):3094, Track Count (by raw appearance):1
Bin:6, Track Count (by log smoothed appearance):364, Track Count (by raw appearance):0
Bin:7, Track Count (by log smoothed appearance):27, Track Count (by raw appearance):0
Bin:8, Track Count (by log smoothed appearance):6, Track Count (by raw appearance):0
Bin:9, Track Count (by log smoothed appearance):3, Track Count (by raw appearance):1


In [52]:
mod_data = {
    'df_track': tracks_full, 
    'df_playlist': data1['df_playlist'], 
    'df_playlist_info': data1['df_playlist_info'], 
    'train_indices': data1['train_indices'], 
    'val_indices': data1['val_indices'], 
    'test_set': data1['test_set']
}

In [53]:
output = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data8.pkl'

In [54]:
pickle.dump(mod_data, open(output, "wb"))

In [45]:
# bins = 5 
# # for i in range(bins): 
# #     print(len(tracks2[tracks2['appear_pop{}'.format(bins)] == i]))
    
# len(tracks2[tracks2['appear_pop{}'.format(bins)] == 0])
# tracks2[tracks2['appear_raw'] > 2]

In [19]:
path = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_Data/datasets/small_10000_100/train_val4.pkl'
data = pickle.load(open(path, "rb"))

In [23]:
tracks = data['df_track']

In [31]:
import numpy as np
bins = 10 
tracks['bin'] = pd.cut(np.log10(tracks['appear_raw']), bins, labels=list(range(bins)), retbins = False, precision=3, ordered=True)
for i in range(bins): 
    print(i, len(tracks[tracks['bin'] == i]), len(tracks[tracks['appear_pop'] == i]))


0 114857 177397
1 34421 1318
2 12947 450
3 7701 163
4 4218 68
5 2651 32
6 1486 19
7 850 6
8 278 5
9 50 1


### OLD 

In [2]:
p5 = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data5.pkl' 
p6 = '/home/mila/r/rebecca.salganik/scratch/MusicSAGE_data_final/LFM_Subset/Smaller_Size/train_val_data6.pkl' 

In [5]:
import pickle 
import pandas as pd

In [6]:
d5 = pickle.load(open(p5, "rb"))

In [8]:
tracks5, users5, interacts5 = d5['df_track'], d5['df_playlist_info'], d5['df_playlist']

In [9]:
d6 = pickle.load(open(p6, "rb"))

In [10]:
tracks6, users6, interacts6 = d6['df_track'], d6['df_playlist_info'], d6['df_playlist']

In [12]:
(users6 == users5).all()

user_id    True
old_pid    True
pid        True
dtype: bool

In [14]:
(interacts6 == interacts5).all()

old_pid    True
old_tid    True
tid        True
pid        True
dtype: bool

In [41]:
interact.groupby('pid')['tid'].apply(len).reset_index(name='#tids').sort_values('#tids')

,pid,#tids
9777,9777,1
9731,9731,1
7989,7989,1
9740,9740,1
8894,8894,1
...,...,...
1573,1573,23440
245,245,23510
558,558,25875
157,157,26328


In [42]:
tracks.appear_norm.describe()

count    8.905680e+05
mean     1.122879e-06
std      6.014289e-06
min      1.370614e-07
25%      1.370614e-07
50%      2.741228e-07
75%      8.223685e-07
max      3.547835e-03
Name: appear_norm, dtype: float64

In [43]:
tracks.appear_raw.describe()

count    890568.000000
mean          8.192523
std          43.880247
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max       25885.000000
Name: appear_raw, dtype: float64

In [44]:
tracks['appear_log'] = tracks['appear_raw'].apply(lambda x: np.log10(x))

In [45]:
tracks['appear_log'].describe()

count    890568.000000
mean          0.499151
std           0.505316
min           0.000000
25%           0.000000
50%           0.301030
75%           0.778151
max           4.413048
Name: appear_log, dtype: float64

In [62]:
pid_in_tid = interact.groupby(['tid'])['pid'].apply(list).reset_index(name='#pids').sort_values('#pids')
pid_in_tid

,tid,#pids
505796,505796,[0]
457166,457166,[0]
852594,852594,[0]
457171,457171,[0]
349867,349867,[0]
...,...,...
396328,396328,[10165]
396331,396331,[10165]
261955,261955,[10166]
261939,261939,[10166]


In [65]:
pid_in_tid['#pids_unique'] = pid_in_tid['#pids'].apply(lambda x: len(set(x)))
pid_in_tid

,tid,#pids,#pids_unique
505796,505796,[0],1
457166,457166,[0],1
852594,852594,[0],1
457171,457171,[0],1
349867,349867,[0],1
...,...,...,...
396328,396328,[10165],1
396331,396331,[10165],1
261955,261955,[10166],1
261939,261939,[10166],1


In [82]:
pid_in_tid['#pids_norm'] = pid_in_tid['#pids_unique']/len(interact.pid.unique())
pid_in_tid
(pid_in_tid['#pids_norm']).describe()

min(pid_in_tid['#pids_norm']) , max(pid_in_tid['#pids_norm'])

(9.83574309039048e-05, 0.09963607750565555)

In [83]:
min(1-pid_in_tid['#pids_norm']) , max(1-pid_in_tid['#pids_norm'])

(0.9003639224943445, 0.9999016425690961)

In [76]:
len(interact.pid.unique())

10167

In [84]:
tracks.columns

Index(['old_tid', 'track_uri', 'track_name', 'album_uri', 'album_name',
       'primary_artist', 'artist_names', 'artist_uris', 'track_name_emb',
       'img_emb', 'genres', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'og_artist_name', 'og_track_name',
       'album_popularity', 'popularity', 'tid', 'genres_vec', 'appear_raw',
       'appear_cat', 'appear_pop', 'appear_norm', 'danceability_10cat',
       'energy_10cat', 'loudness_10cat', 'speechiness_10cat',
       'acousticness_10cat', 'instrumentalness_10cat', 'liveness_10cat',
       'valence_10cat', 'tempo_10cat', 'primary_artist_uri',
       'primary_artist_name', 'arid', 'appear_log'],
      dtype='object')